In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import flammkuchen as fl
import os
import matplotlib.pyplot as plt
import scipy.stats as stats

In [61]:
# load data
data_set_path = Path(r'G:\My Drive\sinff-odor-project\Figuers_data_Marh2023\pseudoPopulationSpikeCount_100sample');
#from os import walk
#mouse_dir_name = next(walk(data_set_path), (None, None, []))[1]  # [] if no file

result_save_path = Path(r"D:\Reza\sniff-odor-progect_final_plots\fig5_encoding_angle_full_space_data")
isExist = os.path.exists(result_save_path)
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs(result_save_path)


In [4]:

#load pseudo population
import mat73
pseudoPopulationSpikeCount_100sample =mat73.loadmat(
    data_set_path / 'pseudoPopulationSpikeCount.mat')

vector = np.vectorize(np.int_)
pseudoPopulationSpikeCount = pseudoPopulationSpikeCount_100sample['pseudoPopulationSpikeCount']
inhTypeId = vector(pseudoPopulationSpikeCount_100sample['inhTypeId'])
concId = vector(pseudoPopulationSpikeCount_100sample['concId'])
odorId = vector(pseudoPopulationSpikeCount_100sample['odorId'])

In [49]:
import math

def angle_ransform(v1, v2):
    cos_angle = np.dot(v1,v2)/np.linalg.norm(v1)/np.linalg.norm(v2) # -> cosine of the angle
    angle_ra = np.arccos(np.clip(cos_angle, -1, 1)) # if you really want the angle
    return abs(90 - math.degrees(angle_ra))

In [54]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve, ShuffleSplit

num_of_sample = 100
num_of_odors = 2

conc_fast_vs_sampling = np.empty((num_of_sample, num_of_odors))
conc_slow_vs_sampling = np.empty((num_of_sample, num_of_odors))
conc_fast_vs_conc_slow = np.empty((num_of_sample, num_of_odors))

clf_fast = LDA(solver='eigen', shrinkage = 'auto');
clf_slow = LDA(solver='eigen', shrinkage = 'auto');
clf_samplin = LDA(solver='eigen', shrinkage = 'auto');

for si in range(num_of_sample):
    for oi in np.arange(1,3):
        sample_data = (pseudoPopulationSpikeCount[:, :, si])

        X_sampling = sample_data[odorId == oi, :]
        y_sampling = inhTypeId[odorId == oi]

        X_fast =  sample_data[(odorId == oi) & (inhTypeId == 1), :]
        y_fast =  concId[(odorId == oi) & (inhTypeId == 1)]

        X_slow =  sample_data[(odorId == oi) & (inhTypeId == 2),  :]
        y_slow =  concId[(odorId == oi) & (inhTypeId == 2)]       


        lda_tranformed_fast = clf_fast.fit(X_fast, y_fast)
        lda_tranformed_slow = clf_slow.fit(X_slow, y_slow)
        lda_inh_type = clf_samplin.fit(X_sampling, y_sampling)

        fast_encoding_axis = clf_fast.scalings_[:, 0]
        slow_encoding_axis = clf_slow.scalings_[:, 0]
        sampling_axis = clf_samplin.scalings_[:, 0]
        
        conc_fast_vs_sampling[si, oi-1] = angle_ransform(fast_encoding_axis, sampling_axis)
        conc_slow_vs_sampling[si, oi-1] = angle_ransform(slow_encoding_axis, sampling_axis)
        conc_fast_vs_conc_slow[si, oi-1] = angle_ransform(fast_encoding_axis, slow_encoding_axis)

In [62]:
from scipy.io import savemat
mdic = {"concVec_Stype_vs_samplingVec": conc_fast_vs_sampling,
        "concVec_Rtype_vs_samplingVec": conc_slow_vs_sampling,
       "concVec_Stype_vs_concVec_Rtype": conc_fast_vs_conc_slow}
savemat(result_save_path / "encoding_angle_full_space_data.mat", mdic)